# PROJETO MC SONAE

## Introdução:

A MC Sonae é um conglomerado português com presença em diversos países, atuando em múltiplos setores da economia, como varejo, imobiliário, tecnologia, entre outros.

Apesar de sua ampla atuação e estrutura organizacional robusta, a empresa enfrenta desafios significativos na gestão de dados entre as diferentes unidades do grupo. Atualmente, os processos de coleta e consolidação de informações são majoritariamente manuais, descentralizados e despadronizados. Essa ausência de uma governança de dados eficaz compromete a consistência das informações, dificultando o planejamento estratégico, a tomada de decisões e a alocação eficiente de investimentos.

## Objetivo:

O principal objetivo do projeto é o desenvolvimento de um sistema de data scraping capaz de coletar, estruturar e padronizar os dados provenientes das diversas áreas e empresas que compõem o grupo MC Sonae.

Esse processo visa:

*   Eliminar a fragmentação e a redundância de dados;
*   Padronizar formatos e estruturas informacionais;
*   Criar uma base de dados unificada, segura e confiável.

A partir dessa infraestrutura consolidada, serão gerados dashboards dinâmicos e interativos, facilitando a visualização e análise dos dados. Esses painéis visuais permitirão aos gestores maior agilidade na interpretação das informações, subsidiando decisões estratégicas baseadas em dados precisos e atualizados.

## Premissas:

Para o desenvolvimento do projeto, foram adotadas algumas premissas:


*   Que os arquivo a serem lidos e tratados estão semiestruturados, ou seja, eles já são fornecidos com alguma estrutura. Para a correta leitura desses arquivos, essa estrutura precisa ser respeitada;


*   O sistema é capaz de ler arquivos nos seguintes formatos: PDF, DOCX, CSV e XLSX. Para os arquivos PDF e DOCX, deve haver apenas dados em forma de texto. Para o formato XLSX e CSV, deve haver dados tabulares (tabelas);


*   Os arquivos tabulares devem já estar formatados com as seguintes colunas: Empresa, Ano, Receita Total (receita bruta), Lucro Líquido, Custo Operacional (OPEX), Número de Funcionários, País e Setor;

## Estrutura do Projeto

O processo de integração de dados consiste em coleta de informações de múltiplas fontes, transformações desses dados para um formato mais adequado e carregamento para um destino final para análise posterior.

As múltiplas fontes de informações são os arquivos de vários formatos que devem estar na pasta `entrada`. Esses dados serão lidos e tratados. Desse processo, serão gerados arquivos gráficos e arquivos provenientes de tratamento de texto e tabela. Os arquivos gráficos serão salvos na pasta `saida_graficos`, com os outros arquivos sendo gravados na pasta `saida`.

Por existirem várias etapas, optou-se por utilizar a metodologia funcional. As várias etapas do processo são divididas em módulos, com cada módulo possuindo funções específicas que executam apenas uma única tarefa específica. Dessa forma, o projeto foi estruturado da seguinte forma:


### `main.py`

Este é "cérebro" de todo o processo. Sua função é gerenciar o fluxo de trabalho, decidir o que precisa ser feito com cada arquivo e chamar os outros módulos para executar as tarefas.



*   Função `processar_arquivo_tabela()`: executa a sequência completa de operações para arquivos de tabela (.csv e .xlsx).


1.   Chama `extrair_tabela` para ler o arquivo e obter um DataFrame;

2.   Passa o DataFrame bruto para o `pipeline_tratamento` para limpeza e padronização;

1.   Usa o DataFrame tratado para chamar `gerar_todos_graficos`;

2.   Chama `salvar_tabela_como_csv` para salvar o resultado em um novo arquivo `.csv`;

1.   Por fim, chama `sumario_executivo` para exibir o resumo dos dados no terminal.

*   Função `processar_arquivo_texto()`: executa a sequência completa de operações para arquivos de texto (.pdf e .docx).

1.  Chama `extrair_texto` para obter uma lista de parágrafos do arquivo;

2.  Aplica a `função limpar_texto` a cada parágrafo;

1.  Agrega os parágrafos limpos para chamar `gerar_estatisticas_texto`.

2.  Monta um dicionário final com as estatísticas e os parágrafos limpos;

1.  Chama `salvar_texto_como_json` para salvar este dicionário em um arquivo `.json`.



*   Função `main()`: é a função que inicia tudo.


1.   Define e cria as pastas `entrada` e `saida`;
2.   Usa `os.listdir()` para obter uma lista com os nomes de todos os arquivos na pasta entrada;


1.   Inicia um laço `for` que percorre cada arquivo da lista;
2.   Dentro do laço, verifica a extensão do arquivo e decide se deve chamar `processar_arquivo_tabela` ou `processar_arquivo_texto`.







In [ ]:
"""
Módulo Principal

Este script é o ponto de entrada do programa. Ele é responsável por:
1. Definir as pastas de entrada e saída de forma segura, baseando-se na localização do próprio script
2. Escanear a pasta de entrada em busca de arquivos para processar
3. Para cada arquivo encontrado, determinar o tipo de processamento necessário com base na sua extensão
4. Chamar as funções de processamento específicas para tabelas (.csv, .xlsx) ou para textos (.pdf, .docx)
"""

import os
from manipulacao_arquivo import extrair_tabela, extrair_texto
from tratamento_dados import pipeline_tratamento
from tratamento_texto import limpar_texto, gerar_estatisticas_texto
from salvar_dados import salvar_tabela_como_csv, salvar_texto_como_json
from sumario import sumario_executivo
from grafico import gerar_todos_graficos

CAMINHO_BASE_DO_SCRIPT = os.path.dirname(os.path.abspath(__file__))
PASTA_ENTRADA = os.path.join(CAMINHO_BASE_DO_SCRIPT, "entrada")
PASTA_SAIDA = os.path.join(CAMINHO_BASE_DO_SCRIPT, "saida")


def processar_arquivo_tabela(caminho_arquivo: str, nome_arquivo: str):
    """
    Executa a pipeline completa para arquivos de tabela (CSV, XLSX).
    """
    # Etapa 1: Extrair os dados brutos da tabela do arquivo
    dado_bruto = extrair_tabela(caminho_arquivo)

    # Se a extração falhar, retorna None e interrompe a função
    if dado_bruto is None:
        return

    # Etapa 2: Aplicar todo o pipeline de tratamento e limpeza dos dados
    dado_tratado = pipeline_tratamento(dado_bruto)

    # Se o tratamento falhar, interrompe a função
    if dado_tratado is None:
        return

    # Etapa 3: Gera e salva os gráficos com base nos dados tratados
    gerar_todos_graficos(dado_tratado)

    # Etapa 4: Preparar o nome e o caminho do arquivo de saída
    nome_base, _ = os.path.splitext(nome_arquivo)
    nome_saida_csv = f"{nome_base}_tratado.csv"
    caminho_saida_csv = os.path.join(PASTA_SAIDA, nome_saida_csv)

    # Etapa 5: Salvar o DataFrame tratado no novo arquivo CSV
    salvar_tabela_como_csv(dado_tratado, caminho_saida_csv)

    # Imprime o sumário dos indicadores financeiros
    sumario_executivo(dado_tratado)


def processar_arquivo_texto(caminho_arquivo: str, nome_arquivo: str):
    """
    Executa a pipeline completa para arquivos de texto (PDF, DOCX)
    """

    # Etapa 1: Extrair o texto do arquivo como uma lista de parágrafos
    lista_paragrafos_brutos = extrair_texto(caminho_arquivo)

    if not lista_paragrafos_brutos:
        print("   - Falha ao extrair texto ou arquivo sem conteúdo. Pulando para o próximo arquivo.")
        return

    # Etapa 2: Limpar cada parágrafo individualmente
    print("   - Limpando cada parágrafo extraído...")
    paragrafos_limpos = []
    for p in lista_paragrafos_brutos:
        paragrafo_processado = limpar_texto(p)
        paragrafos_limpos.append(paragrafo_processado)

    paragrafos_realmente_limpos = []
    for p in paragrafos_limpos:
        if p:
            paragrafos_realmente_limpos.append(p)
    paragrafos_limpos = paragrafos_realmente_limpos

    # Etapa 3: Gerar estatísticas a partir do texto completo
    print("   - Gerando estatísticas gerais do texto...")
    texto_completo_limpo = " ".join(paragrafos_limpos)
    estatisticas = gerar_estatisticas_texto(texto_completo_limpo)

    # Etapa 4: Montar a estrutura do dicionário para salvar no JSON
    dados_finais = {
        "estatisticas_gerais": estatisticas,
        "paragrafos_limpos": paragrafos_limpos
    }

    # Etapa 5: Preparar o nome e o caminho do arquivo de saída JSON
    nome_base, _ = os.path.splitext(nome_arquivo)
    nome_saida_json = f"{nome_base}_texto.json"
    caminho_saida_json = os.path.join(PASTA_SAIDA, nome_saida_json)

    # Etapa 6: Salvar o dicionário final no novo arquivo JSON
    salvar_texto_como_json(dados_finais, caminho_saida_json)

def main():
    """
    Função principal que inicia e gerencia todo o processo
    """
    print("="*60)
    print(" " * 20 + "INICIANDO PROCESSAMENTO")
    print("="*60)

    os.makedirs(PASTA_ENTRADA, exist_ok=True)
    os.makedirs(PASTA_SAIDA, exist_ok=True)

    try:
        lista_arquivos = os.listdir(PASTA_ENTRADA)
        if not lista_arquivos:
            print(f"\nAVISO: A pasta '{PASTA_ENTRADA}' está vazia.")
            return

    except FileNotFoundError:
        print(f"ERRO: A pasta de entrada '{PASTA_ENTRADA}' não foi encontrada!")
        return

    print(f"\nEncontrados {len(lista_arquivos)} arquivo(s) na pasta de entrada.")

    for nome_arquivo in lista_arquivos:
        caminho_completo = os.path.join(PASTA_ENTRADA, nome_arquivo)
        _, extensao = os.path.splitext(nome_arquivo)
        extensao = extensao.lower()

        print(f"\n--- Processando arquivo: '{nome_arquivo}' ---")

        if extensao in ['.csv', '.xlsx']:
            processar_arquivo_tabela(caminho_completo, nome_arquivo)

        elif extensao in ['.pdf', '.docx']:
            processar_arquivo_texto(caminho_completo, nome_arquivo)

        else:
            print(f"   - AVISO: Extensão '{extensao}' não suportada.")

    print("\n" + "="*60)
    print(" " * 17 + "PROCESSAMENTO FINALIZADO")
    print("="*60)


if __name__ == "__main__":
    main()

### `manipulacao_arquivo.py`

Este módulo é responsável pela primeira etapa: Extrair (o "E" de ETL). Ele sabe como abrir e ler os diferentes formatos de arquivo.



*   Função `extrair_tabela()`: lê um arquivo `.csv` ou `.xlsx` e o converte em uma tabela de dados (DataFrame). Utiliza a biblioteca pandas `pd.read_csv` e `pd.read_excel`, que é a ferramenta padrão em Python para manipulação de tabelas.




*   Função `extrair_texto()`: lê um arquivo `.pdf` ou `.docx` e extrai todo o seu conteúdo textual, retornando-o como uma lista de parágrafos. Para `.pdf`, usa a biblioteca pdfplumber para extrair o texto de cada página e depois o divide por quebras de linha. Para `.docx`, usa a biblioteca `python-docx` que já consegue identificar e extrair cada parágrafo individualmente.



In [ ]:
"""
Módulo de Manipulação de Arquivos (Extração)

Este módulo contém as funções responsáveis por ler os diferentes tipos de arquivos
(.csv, .xlsx, .pdf, .docx) e extrair seu conteúdo bruto.
Ele separa a lógica de extração de tabelas e de textos em funções diferentes.
"""

import os
import pdfplumber
import pandas as pd
from docx import Document

def extrair_tabela(caminho_arquivo: str) -> pd.DataFrame:
    """
    Extrai uma tabela de arquivos .csv ou .xlsx
    """
    print(f"   - Tentando extrair TABELA de '{os.path.basename(caminho_arquivo)}'...")

    # Extrai a extensão do arquivo para determinar como lê-lo
    _, extensao = os.path.splitext(caminho_arquivo)
    extensao = extensao.lower()

    # Bloco try/except para capturar possíveis erros durante a leitura do arquivo
    try:
        if extensao == '.csv':
            # Se for CSV, usa a função read_csv do Pandas.
            df = pd.read_csv(caminho_arquivo)
        elif extensao == '.xlsx':
            # Se for Excel, usa a função read_excel do Pandas.
            df = pd.read_excel(caminho_arquivo)
        else:
            # Se a extensão não for suportada para tabelas, informa e retorna None.
            print(f"   - ERRO: Extensão '{extensao}' não é suportada para extração de tabelas.")
            return None

        print("     -> Tabela extraída com sucesso.")
        # Retorna o DataFrame criado.
        return df

    except Exception as e:
        # Se qualquer outro erro ocorrer, imprime o erro e retorna None
        print(f"   - ERRO ao ler o arquivo de tabela: {e}")
        return None

def extrair_texto(caminho_arquivo: str) -> list[str]:
    """
    Extrai o texto corrido de arquivos .pdf ou .docx, parágrafo por parágrafo
    """
    print(f"   - Tentando extrair TEXTO de '{os.path.basename(caminho_arquivo)}'...")

    _, extensao = os.path.splitext(caminho_arquivo)
    extensao = extensao.lower()
    lista_paragrafos = [] # Inicializa uma lista vazia para armazenar os parágrafos.

    try:
        if extensao == '.pdf':
            # Usa a biblioteca pdfplumber para abrir e ler o PDF.
            with pdfplumber.open(caminho_arquivo) as pdf:
                # Corre cada página do documento.
                for pagina in pdf.pages:
                    texto_pagina = pagina.extract_text()
                    # Se algum texto for extraído da página
                    if texto_pagina:
                        # divide o texto da página por quebras de linha ('\n') para simular parágrafos
                        paragrafos_pagina = texto_pagina.split('\n')
                        # Adiciona os "parágrafos" encontrados à lista principal
                        lista_paragrafos.extend(paragrafos_pagina)

        elif extensao == '.docx':
            # Usa a biblioteca python-docx para abrir o documento Word
            documento = Document(caminho_arquivo)
            # A biblioteca já fornece uma lista de parágrafos
            for paragrafo in documento.paragraphs:
                # Adiciona o texto de cada parágrafo à lista
                lista_paragrafos.append(paragrafo.text)

        else:
            # Se a extensão não for suportada para textos, informa e retorna None
            print(f"   - ERRO: Extensão '{extensao}' não é suportada para extração de texto.")
            return None

        # Limpa a lista, removendo itens vazios ou que contêm apenas espaços
        paragrafos_filtrados = [] # Cria uma lista vazia para o resultado
        for p in lista_paragrafos: # Corre cada parágrafo extraído
            # A condição 'if p' verifica se a string não é vazia.
            # A condição 'not p.isspace()' verifica se a string não contém apenas espaços em branco
            if p and not p.isspace():
                # Remove espaços em branco do início e do fim do parágrafo
                paragrafos_filtrados.append(p.strip())


        # Se a lista final de parágrafos não estiver vazia
        if paragrafos_filtrados:
            print(f"     -> Texto extraído com sucesso. {len(paragrafos_filtrados)} parágrafos encontrados.")
            # Retorna a lista de parágrafos limpos.
            return paragrafos_filtrados
        else:
            # Se, após a filtragem, a lista estiver vazia, informa e retorna None
            print("   - AVISO: Nenhum texto foi encontrado no arquivo.")
            return None

    except Exception as e:
        # Se qualquer erro ocorrer durante o processo, imprime a mensagem e retorna None.
        print(f"   - ERRO ao extrair texto do arquivo: {e}")
        return None

### `tratamento_dados.py`

Este módulo é responsável pela segunda etapa para dados tabulares: Transformar (o "T" de ETL). Sua missão é pegar a tabela bruta e deixá-la limpa, padronizada e pronta para análise.



*   Função `remover_duplicadas()`: remove linhas que são cópias exatas de outras;


*   Função `converter_tipos_colunas()`: garante que colunas que deveriam ser numéricas (como 'Ano', 'Lucro Líquido') sejam de fato tratadas como números, e não como texto;
*   Função `tratar_dados_nulos()`: encontra células vazias na tabela e as preenche com valores padrão (0 para números, 'Não Informado' para textos);


*   Função `padronizar_texto()`: padroniza textos para manter a consistência.
*   Função `pipeline_tratamento()`: orquestra todas as funções de tratamento, executando as funções de limpeza na sequência correta.


In [ ]:
"""
Módulo de Tratamento de Dados de Tabela

Este módulo contém todas as funções necessárias para limpar e transformar os dados
extraídos em formato de tabela (DataFrame do Pandas).
As operações incluem remoção de duplicatas, conversão de tipos,
tratamento de valores nulos e padronização de textos.
"""

import pandas as pd

def remover_duplicadas(dados: pd.DataFrame) -> pd.DataFrame:
    """
    Verifica e remove linhas duplicadas de um DataFrame.
    """
    # Conta o número de linhas que são duplicatas exatas.
    num_duplicatas = dados.duplicated().sum()

    # Se encontrar uma ou mais duplicatas
    if num_duplicatas > 0:
        print(f"\n---Removendo {num_duplicatas} linhas duplicadas ...")
        # Remove as duplicatas, mantendo a primeira ocorrência e rearruma o índice do DataFrame após a remoção
        dados_sem_duplicatas = dados.drop_duplicates(keep='first').reset_index(drop=True)
        return dados_sem_duplicatas
    # Caso contrário
    else:
        print("\n---Não foram encontradas linhas duplicadas.")
        return dados

def converter_tipos_colunas(dados: pd.DataFrame) -> pd.DataFrame:
    """
    Converte colunas específicas para o tipo numérico.
    """
    print("\n---Convertendo tipos de dados das colunas ...")

    # Lista de colunas que esperamos que sejam numéricas
    colunas_numericas = ['Ano', 'Receita Total (receita bruta)', 'Lucro Líquido',
                         'Custo Operacional (OPEX)', 'Número de Funcionários']

    # Varr cada nome de coluna na lista.
    for coluna in colunas_numericas:
        # Verifica se a coluna realmente existe no DataFrame.
        if coluna in dados.columns:
            # Converte a coluna para tipo numérico
            # 'errors=coerce' força valores que não podem ser convertidos a se tornarem 'NaN' (Not a Number)
            dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')
        else:
            # Se a coluna não for encontrada, imprime um aviso
            print(f"\n---Coluna numérica '{coluna}' não encontrada")

    return dados

def tratar_dados_nulos(dados: pd.DataFrame) -> pd.DataFrame:
    """
    Preenche valores nulos (NaN) em colunas numéricas e de texto
    """
    print("\n---Tratando dados nulos/em branco ...")

    # Para colunas numéricas, preenche os valores nulos com 0
    colunas_numericas = dados.select_dtypes(include=['number']).columns
    dados[colunas_numericas] = dados[colunas_numericas].fillna(0)

    # Para colunas de texto, preenche os valores nulos com 'Não Informado'
    colunas_texto = dados.select_dtypes(include=['object']).columns
    dados[colunas_texto] = dados[colunas_texto].fillna('Não Informado')

    # Garante que colunas que devem ser inteiras (sem casas decimais) sejam convertidas
    colunas_inteiras = ['Ano', 'Número de Funcionários']
    for coluna in colunas_inteiras:
        if coluna in dados.columns:
            dados[coluna] = dados[coluna].astype(int)

    return dados

def padronizar_texto(dados: pd.DataFrame) -> pd.DataFrame:
    """
    Padroniza o texto de colunas específicas para o formato "Title Case" - Primeira letra maiúscula e o resto minúscula
    """
    print("\n---Padronizando colunas de texto ...")

    # Lista de colunas de texto a serem padronizadas.
    colunas_para_padronizar = ['Empresa', 'País', 'Setor']
    for coluna in colunas_para_padronizar:
        dados[coluna] = dados[coluna].str.title()

    return dados

def pipeline_tratamento(dados: pd.DataFrame) -> pd.DataFrame:
    """
    Orquestra a execução de todas as funções de tratamento em sequência.
    """
    # Caso de um DataFrame vazio seja passado
    if dados is None:
        return None

    print("\n---Iniciando pipeline de tratamento de dados---")

    # Executa cada etapa do tratamento na ordem definida.
    dados_tratados = converter_tipos_colunas(dados)
    dados_tratados = tratar_dados_nulos(dados_tratados)
    dados_tratados = padronizar_texto(dados_tratados)
    dados_tratados = remover_duplicadas(dados_tratados)

    # Retorna o DataFrame final.
    return dados_tratados

### `tratamento_texto.py`

Este módulo também faz a etapa de Transformação, mas com foco nos textos extraídos dos PDFs e DOCX.





*   Função `limpar_texto()`: pega um parágrafo de texto e realiza uma limpeza básica.Converte tudo para minúsculas e remove sinais de pontuação para facilitar a análise computacional.
*   Função `gerar_estatisticas_texto()`: calcula métricas simples sobre o texto. Conta o número total de palavras, o número de palavras únicas e usa a classe `Counter` para encontrar as 5 palavras mais comuns no documento.



In [ ]:
"""
Módulo de Tratamento de Texto

Este módulo contém funções para processamento e análise de texto.
As operações incluem limpeza (remoção de pontuação, normalização de caixa)
e a geração de estatísticas básicas, como contagem de palavras.
"""

import re
from collections import Counter
import string

def limpar_texto(texto: str) -> str:
    """
    Realiza uma limpeza básica no texto.
    """
    # Etapa 1: Converte toda a string para letras minúsculas
    texto = texto.lower()

    # Etapa 2: Remove todos os caracteres de pontuação.
    # 'string.punctuation' é uma string que contém todos os sinais de pontuação comuns.
    # O método 'translate' é uma forma eficiente de remover múltiplos caracteres de uma vez.
    texto = texto.translate(str.maketrans('', '', string.punctuation))

    # Etapa 3: Normaliza os espaços em branco.
    # 'texto.split()' quebra a string em uma lista de palavras (removendo espaços extras)
    # "' '.join(...)" junta a lista de volta em uma string, com um único espaço entre as palavras
    texto = ' '.join(texto.split())

    return texto

def gerar_estatisticas_texto(texto_limpo: str) -> dict:
    """
    Gera um dicionário com estatísticas básicas sobre o texto
    """
    # Quebra a string de texto em uma lista de palavras
    palavras = texto_limpo.split()

    # Calcula o número total de palavras na lista
    total_palavras = len(palavras)

    # 'set(palavras)' cria um conjunto, que automaticamente remove duplicatas.
    # 'len()' então nos dá o número de palavras únicas.
    palavras_unicas = len(set(palavras))

    # 'Counter' é uma classe especializada que conta a frequência de cada item em uma lista
    frequencia_palavras = Counter(palavras)
    # '.most_common(5)' retorna uma lista de tuplas com as 5 palavras mais frequentes e suas contagens
    top_5_palavras = frequencia_palavras.most_common(5)

    # Monta o dicionário final com os resultados
    estatisticas = {
        "total_de_palavras": total_palavras,
        "total_de_palavras_unicas": palavras_unicas,
        "top_5_palavras_mais_comuns": top_5_palavras
    }

    return estatisticas

### `salvar_dados.py`

Este módulo é responsável pela etapa final: Carregar (o "L" de ETL). Ele pega os dados já processados e os salva em disco.


*   Função `salvar_tabela_como_csv()`: pega o DataFrame tratado e o salva em um novo arquivo no formato `.csv`.
*   Função `salvar_texto_como_json()`: pega o dicionário com as estatísticas e os parágrafos limpos e o salva em um arquivo no formato `.json`.



In [ ]:
"""
Módulo de Salvamento de Dados (Carregamento)

Este módulo contém as funções para persistir os dados processados em disco.
Ele lida com o salvamento de dados em forma de tabela, no formato .csv, e dados de texto, no formato .json
"""

import pandas as pd
import json
import os

def salvar_tabela_como_csv(dados: pd.DataFrame, caminho_saida: str):
    """
    Salva um DataFrame em um arquivo no formato .csv
    """
    # Verifica se os dados estiverem vazios
    if dados is None:
        print("   - AVISO: Nenhum dado de tabela para salvar.")
        return

    print(f"   - Salvando tabela tratada em '{caminho_saida}'...")
    try:
        # Garante que a pasta de destino exista antes de tentar salvar.
        os.makedirs(os.path.dirname(caminho_saida), exist_ok=True)

        # 'index=False' impede o Pandas de salvar o índice do DataFrame como uma coluna no CSV
        # 'encoding='utf-8-sig'' garante a compatibilidade com acentos e caracteres especiais ao abrir o arquivo no Excel
        dados.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
        print(f"     -> Tabela salva com sucesso.")

    except Exception as e:
        print(f"   - ERRO ao salvar o arquivo CSV: {e}")


def salvar_texto_como_json(dados_para_salvar: dict, caminho_saida: str):
    """
    Salva um dicionário de dados em um arquivo no formato .json.
    """
    # Não faz nada se o dicionário estiver vazio
    if not dados_para_salvar:
        print("   - AVISO: Nenhum dado de texto para salvar.")
        return

    print(f"   - Salvando dados de texto em '{caminho_saida}'...")
    try:
        # Garante a existência do diretório
        os.makedirs(os.path.dirname(caminho_saida), exist_ok=True)

        # Abre o arquivo em modo de escrita ('w') com a codificação UTF-8.
        with open(caminho_saida, 'w', encoding='utf-8') as f:
            # 'json.dump' escreve o dicionário no arquivo.
            # 'ensure_ascii=False' permite que caracteres acentuados sejam salvos corretamente.
            # 'indent=4' formata o JSON de forma legível, com 4 espaços de indentação.
            json.dump(dados_para_salvar, f, ensure_ascii=False, indent=4)
        print(f"     -> Arquivo JSON salvo com sucesso.")

    except Exception as e:
        print(f"   - ERRO ao salvar o arquivo JSON: {e}")

### `grafico.py`

Este módulo é focado em criar representações visuais dos dados das tabelas para facilitar a compreensão. É uma função genérica que cria um único gráfico de barras e o salva como uma imagem `.png`, utilizando a biblioteca `matplotlib` para desenhar o gráfico, customizar títulos, eixos e cores, e finalmente salvá-lo em um arquivo.



*   Funções `gerar_todos_graficos()`: orquestra a criação de todos os gráficos de análise do projeto. Prepara os dados (agrupando por empresa e por país) e chama a função `criar_e_salvar_grafico_barras` várias vezes, uma para cada gráfico que se deseja criar.

In [ ]:
"""
Módulo de Geração de Gráficos

Este módulo utiliza a biblioteca Matplotlib para criar visualizações dos dados
tabulares tratados. As funções aqui geram gráficos de barras e os salvam
como arquivos de imagem .png.
"""

import os
import pandas as pd
import matplotlib.pyplot as plt

def criar_e_salvar_grafico_barras(dados: pd.DataFrame, eixo_x: str, eixo_y: str, titulo: str, pasta_saida: str):
    """
    Cria e salva um único gráfico de barras como um arquivo de imagem (.png)
    """
    # Se não houver dados, não faz nada.
    if dados is None or dados.empty:
        print(f"   - Aviso: Não há dados para gerar o gráfico '{titulo}'.")
        return

    print(f"   - Gerando e salvando gráfico: '{titulo}'...")

    try:
        # Garante que a pasta de saída para os gráficos exista
        os.makedirs(pasta_saida, exist_ok=True)

        # Ordena os dados pelo eixo Y em ordem decrescente para um melhor visual
        dados_ordenados = dados.sort_values(by=eixo_y, ascending=False)

        # Cria uma nova figura/gráfico com um tamanho específico
        plt.figure(figsize=(12, 7))

        # Plota os dados como um gráfico de barras
        plt.bar(dados_ordenados[eixo_x], dados_ordenados[eixo_y])

        # Define os textos do gráfico (título, rótulos dos eixos)
        plt.title(titulo, fontsize=16)
        plt.xlabel(eixo_x, fontsize=12)
        plt.ylabel(eixo_y, fontsize=12)

        # Rotaciona os rótulos do eixo X para evitar sobreposição
        plt.xticks(rotation=45, ha='right')

        # Adiciona uma grade horizontal para facilitar a leitura
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        # Ajusta o layout para garantir que nada seja cortado
        plt.tight_layout()

        # Cria um nome de arquivo a partir do título do gráfico
        nome_arquivo = f"{titulo.lower().replace(' ', '_')}.png"
        caminho_completo = os.path.join(pasta_saida, nome_arquivo)

        # Salva a figura gerada no caminho especificado
        plt.savefig(caminho_completo)

        # Fecha a figura salva
        plt.close()

        print(f"     -> Gráfico salvo em '{caminho_completo}'")

    except Exception as e:
        print(f"   - ERRO ao gerar o gráfico '{titulo}': {e}")


def gerar_todos_graficos(dados: pd.DataFrame):
    """
    Orquestra a criação de todos os gráficos de análise definidos.
    """
    # Se não houver dados, interrompe o processo
    if dados is None:
        print("\n--- Análise gráfica interrompida: DataFrame está vazio. ---")
        return

    # Define o caminho para a pasta de saída dos gráficos de forma segura
    CAMINHO_BASE_DO_SCRIPT = os.path.dirname(os.path.abspath(__file__))
    PASTA_GRAFICOS = os.path.join(CAMINHO_BASE_DO_SCRIPT, "saida_graficos")

    print(f"\n--- Iniciando geração de gráficos (salvando em '{PASTA_GRAFICOS}') ---")

    # Agrupa os dados por empresa e soma os valores numéricos
    dados_por_empresa = dados.groupby('Empresa').sum(numeric_only=True).reset_index()

    # Chama a função de criação de gráficos para cada análise de empresa
    criar_e_salvar_grafico_barras(dados_por_empresa, 'Empresa', 'Receita Total (receita bruta)', 'Receita Total por Empresa', PASTA_GRAFICOS)
    criar_e_salvar_grafico_barras(dados_por_empresa, 'Empresa', 'Lucro Líquido', 'Lucro Líquido por Empresa', PASTA_GRAFICOS)

    # Agrupa os dados por país e soma os valores numéricos
    dados_por_pais = dados.groupby('País').sum(numeric_only=True).reset_index()

    # Chama a função de criação de gráficos para cada análise de país
    criar_e_salvar_grafico_barras(dados_por_pais, 'País', 'Receita Total (receita bruta)', 'Receita Total por País', PASTA_GRAFICOS)
    criar_e_salvar_grafico_barras(dados_por_pais, 'País', 'Lucro Líquido', 'Lucro Líquido por País', PASTA_GRAFICOS)

    print("--- Geração de gráficos finalizada ---")

### sumario.py

Este módulo tem o objetivo de apresentar os resultados financeiros da análise das tabelas de forma rápida e direta no terminal do usuário.



*   Função `sumario_executivo()`: calcula totais e outros indicadores-chave e os exibe em um formato legível. Soma os valores de colunas financeiras importantes (como 'Receita Total' e 'Lucro Líquido') e conta quantas empresas únicas foram analisadas, imprimindo tudo em um bloco de texto bem formatado.



In [ ]:
"""
Módulo de Geração de Sumário Executivo

Este módulo é responsável por calcular e exibir um resumo dos principais
indicadores financeiros e operacionais das tabelas tratadas
"""

import pandas as pd

def sumario_executivo(dados: pd.DataFrame):
    """
    Calcula e imprime no console um sumário dos dados.
    """
    # Se não houver dados, imprime uma mensagem e encerra
    if dados is None or dados.empty:
        print("\n--- Não há dados para gerar o sumário. ---")
        return

    # Imprime um cabeçalho para o sumário
    print("\n" + "="*50)
    print(" " * 15 + "SUMÁRIO EXECUTIVO")
    print("="*50)

    try:
        # Lista de colunas financeiras que queremos somar.
        colunas_financeiras = [
            'Receita Total (receita bruta)',
            'Lucro Líquido'
        ]

        # Verifica se uma coluna esperada (colunas_financeiras) existe no arquivo
        colunas_existentes = []
        for col in colunas_financeiras:
            if col in dados.columns:
                colunas_existentes.append(col)

        # Calcula a soma apenas para as colunas que foram encontradas
        somas = dados[colunas_existentes].sum()

        # Conta o número de valores únicos na coluna 'Empresa'
        num_empresas = dados['Empresa'].nunique()

        # Imprime o resumo dos resultados.
        print(f"Indicadores Consolidados para {num_empresas} empresa(s):")

        # Corre os resultados das somas para exibi-los.
        for nome_coluna, total in somas.items():
            # Formatação da string para alinhar os valores e formatar os números.
            # ':<30' alinha o texto à esquerda em um espaço de 30 caracteres.
            # ':,.2f' formata o número com separador de milhar e duas casas decimais
            print(f"  - {nome_coluna:<30}: € {total:,.2f}")

    except Exception as e:
        # Se ocorrer qualquer erro durante os cálculos, ele será capturado e exibido.
        print(f"\n--- ERRO ao gerar o sumário executivo: {e} ---")

    finally:
        # Este bloco é sempre executado, imprimindo o rodapé do sumário.
        print("="*50)